In [2]:
import os

from dotenv import load_dotenv
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents.agent_toolkits.pandas.base import _get_functions_prompt_and_tools
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.agents import AgentExecutor, AgentType
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory



import pandas as pd


load_dotenv('../.flaskenv')
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [3]:
embeddings = OpenAIEmbeddings()
persist_directory = '/home/fasih/ADIB/chroma3'

adib = pd.read_csv('~/ADIB/data/ADIB.csv')

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
vectordb._collection.count()

2502

In [4]:
prefix = ("You are working with a pandas dataframe in Python."
          "Always filter data if needed."
          "The name of the dataframe is `adib`"
          "make sure to use search_adib_annual_report for the columns that are not present in adib dataset"
         )


suffix = """
This is the result of `print(df.head())`:
{df_head}
"""

suffix_to_use = suffix.format(df_head=str(adib.head(1).to_markdown()))



system_message = SystemMessage(content=prefix+suffix_to_use)

MEMORY_KEY = "chat_history" 
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

In [5]:
df_tool = PythonAstREPLTool(
    locals={"adib":adib}, 
    description=("A Python shell. Use this to execute python commands."
                 "Input should be a valid python command. When using this tool, sometimes output is abbreviated"
                 "- make sure it does not look abbreviated before using it in your answer."
                 "You have CSV data for ADIB balance sheet as pandas dataframe with name adib."
                 "Numbers in csv are in AED Millions"
                )
)

retriever_tool = create_retriever_tool(
    vectordb.as_retriever(),
    "search_adib_annual_report",
    "Searches and returns the parts of document from Abu Dhabi Islamic Bank's annual report."
)

In [6]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [14]:
agent = OpenAIFunctionsAgent(
            llm=llm,
            prompt=prompt,
            tools=[df_tool,retriever_tool]
)

In [15]:
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)
memory2 = ConversationBufferWindowMemory(memory_key=MEMORY_KEY, return_messages=True, k=4)


In [16]:
agent = AgentExecutor(
    agent=agent,
    tools=[df_tool,retriever_tool],
    memory=memory2,
    verbose=True
)

In [20]:
agent.run('who are the best tekken players? and why? also answer in german')



> Entering new AgentExecutor chain...
Die besten Tekken-Spieler können je nach Zeitraum und dem spezifischen Tekken-Spiel variieren. Es gibt jedoch mehrere Spieler, die aufgrund ihrer konstanten Leistung, Turniergewinne und ihres hohen Spielniveaus weithin als einige der besten Tekken-Spieler angesehen werden. Hier sind einige bemerkenswerte Tekken-Spieler:

1. Knee: Knee, auch bekannt als Jae Min Bae, gilt als einer der größten Tekken-Spieler aller Zeiten. Er ist seit vielen Jahren eine dominierende Kraft in der Tekken-Wettkampfszene und hat zahlreiche Turniere gewonnen. Knee ist bekannt für seine außergewöhnliche Ausführung, Anpassungsfähigkeit und Beherrschung mehrerer Charaktere.

2. Arslan Ash: Arslan Ash aus Pakistan erlangte internationale Anerkennung, nachdem er das Tekken 7-Turnier bei Evo Japan 2019 gewonnen hatte. Er zeigte unglaubliches Können und Wissen über das Spiel und besiegte Top-Spieler aus der ganzen Welt. Arslan Ash ist bekannt für seine außergewöhnliche Verteidi

'Die besten Tekken-Spieler können je nach Zeitraum und dem spezifischen Tekken-Spiel variieren. Es gibt jedoch mehrere Spieler, die aufgrund ihrer konstanten Leistung, Turniergewinne und ihres hohen Spielniveaus weithin als einige der besten Tekken-Spieler angesehen werden. Hier sind einige bemerkenswerte Tekken-Spieler:\n\n1. Knee: Knee, auch bekannt als Jae Min Bae, gilt als einer der größten Tekken-Spieler aller Zeiten. Er ist seit vielen Jahren eine dominierende Kraft in der Tekken-Wettkampfszene und hat zahlreiche Turniere gewonnen. Knee ist bekannt für seine außergewöhnliche Ausführung, Anpassungsfähigkeit und Beherrschung mehrerer Charaktere.\n\n2. Arslan Ash: Arslan Ash aus Pakistan erlangte internationale Anerkennung, nachdem er das Tekken 7-Turnier bei Evo Japan 2019 gewonnen hatte. Er zeigte unglaubliches Können und Wissen über das Spiel und besiegte Top-Spieler aus der ganzen Welt. Arslan Ash ist bekannt für seine außergewöhnliche Verteidigung, solide Grundlagen und die Fäh

In [12]:
agent.run("Which operating expenses have unexpectedly increased in the year 2022 compared to the previous three years?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `adib[['Periods', 'Operating expenses']].sort_values('Periods', ascending=False).head(4)`


    Periods  Operating expenses
11     2028             2006.62
10     2027             2062.76
9      2026             2118.90
8      2025             2175.05
Invoking: `search_adib_annual_report` with `operating expenses`


[Document(page_content='and resale of properties and all associated activities.\nOther operations - Other operations comprises mainly of Head Office, subsidiaries, associates and joint ventures other than above categories \nincluding unallocated costs.\nManagement monitors the operating results of the operating segments separately for the purpose of making decisions about resource allocation \nand performance assessment. Segment performance is evaluated based on operating profit or loss.\nBusiness segments information for the year ended 31 December 2018 were as follows:\nGlobal Retail\nbanking\nAED ‘0

'The operating expenses that have unexpectedly increased in the year 2022 compared to the previous three years are as follows:\n\n1. Global Retail banking: The operating expenses increased from AED 1,205,586 in 2021 to AED 1,809,154 in 2022.\n\n2. Global Wholesale banking: The operating expenses increased from AED 17,402 in 2021 to AED 275,499 in 2022.\n\n3. Private banking: The operating expenses increased from AED 81,167 in 2021 to AED 69,725 in 2022.\n\n4. Treasury: The operating expenses increased from AED 740,335 in 2021 to AED 598,830 in 2022.\n\n5. Real estate: The operating expenses increased from AED 3,428 in 2021 to AED 6,020 in 2022.\n\n6. Other operations: The operating expenses increased from AED 297,199 in 2021 to AED 594,051 in 2022.\n\nThese segments have experienced unexpected increases in operating expenses in the year 2022 compared to the previous three years.'

In [173]:
agent.run('What could be the potential strategies to minimize the costs incurred?')



> Entering new AgentExecutor chain...
To minimize costs incurred, here are some potential strategies that can be considered:

1. Review and optimize operational processes: Identify areas where processes can be streamlined or automated to reduce manual effort and associated costs. Look for opportunities to eliminate unnecessary steps, improve efficiency, and reduce waste.

2. Negotiate better supplier contracts: Review existing supplier contracts and negotiate better terms, including discounts, bulk purchasing options, or longer payment terms. Consider exploring alternative suppliers to ensure competitive pricing.

3. Implement cost control measures: Implement strict cost control measures across the organization. This can include setting spending limits, requiring approval for non-essential expenses, and regularly monitoring and analyzing expenses to identify areas of overspending.

4. Encourage cost-conscious culture: Foster a culture of cost-consciousness among employees. Educate an

"To minimize costs incurred, here are some potential strategies that can be considered:\n\n1. Review and optimize operational processes: Identify areas where processes can be streamlined or automated to reduce manual effort and associated costs. Look for opportunities to eliminate unnecessary steps, improve efficiency, and reduce waste.\n\n2. Negotiate better supplier contracts: Review existing supplier contracts and negotiate better terms, including discounts, bulk purchasing options, or longer payment terms. Consider exploring alternative suppliers to ensure competitive pricing.\n\n3. Implement cost control measures: Implement strict cost control measures across the organization. This can include setting spending limits, requiring approval for non-essential expenses, and regularly monitoring and analyzing expenses to identify areas of overspending.\n\n4. Encourage cost-conscious culture: Foster a culture of cost-consciousness among employees. Educate and train employees on the import

In [13]:
result = agent.run('Which components of the income statement, balance sheet, and cash flows have exhibited irregular trends over the past three years?')
result



> Entering new AgentExecutor chain...

Invoking: `search_adib_annual_report` with `{'search_term': 'income statement'}`


[Document(page_content='Financial Statements Abridged Income StatementFor the year ended  \n31 December\n2020\nAED Million2021\nAED Million\nFees and commissions 45.1 45.4\nOther income  -    0.3 \nTotal Revenues 45.1 45.7 \nTotal expenses (48.9) (44.0)\n(Loss) / Profit for the year (3.8) 1.8 \n NON-FINANCIAL SERVICES \nBUSINESS REVIEW  \nCONTINUED\nADIB Annual Report 2021\n38', metadata={'page': 37, 'source': '/home/fasih/ADIB/annual_report/ADIB Annual Report 2021.pdf'}), Document(page_content='Financial StatementsAbridged Income StatementFor the year ended  \n31 December\n2021\nAED Million2022\nAED Million\nFees and commissions 45.4 46.4\nOther income  0.3  - \nTotal Revenues 45.7 46.4 \nTotal expenses (44.0) (41.1)\n(Loss) / Profit for the year 1.8 5.3 \n NON-FINANCIAL SERVICES \nBUSINESS REVIEW \nCONTINUED\nOPERATIONAL \nREVIEW \nRetail banking group \nWholesa

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4864 tokens (4725 in the messages, 139 in the functions). Please reduce the length of the messages or functions.

In [175]:
result = agent.run('What are the possible ways to minimize operating expenses for each operating segment')
result



> Entering new AgentExecutor chain...
To minimize operating expenses for each operating segment, here are some possible ways:

1. Customer Financing Segment:
   - Streamline loan origination and approval processes to reduce administrative costs.
   - Implement credit risk assessment tools and models to improve loan quality and reduce provisions for impairment.
   - Offer digital self-service options for customers to reduce the need for manual assistance and lower operational costs.
   - Optimize collection processes to minimize delinquencies and associated costs.

2. Investments Segment:
   - Utilize technology platforms and automated tools for investment research and analysis to reduce manual effort and associated costs.
   - Optimize portfolio management strategies to minimize transaction costs and improve investment returns.
   - Leverage data analytics to identify investment opportunities with higher potential returns and lower associated costs.
   - Regularly review and rebalanc

"To minimize operating expenses for each operating segment, here are some possible ways:\n\n1. Customer Financing Segment:\n   - Streamline loan origination and approval processes to reduce administrative costs.\n   - Implement credit risk assessment tools and models to improve loan quality and reduce provisions for impairment.\n   - Offer digital self-service options for customers to reduce the need for manual assistance and lower operational costs.\n   - Optimize collection processes to minimize delinquencies and associated costs.\n\n2. Investments Segment:\n   - Utilize technology platforms and automated tools for investment research and analysis to reduce manual effort and associated costs.\n   - Optimize portfolio management strategies to minimize transaction costs and improve investment returns.\n   - Leverage data analytics to identify investment opportunities with higher potential returns and lower associated costs.\n   - Regularly review and rebalance investment portfolios to 

In [178]:
result = agent.run('what cost heads do I have?')
result



> Entering new AgentExecutor chain...

Invoking: `search_adib_annual_report` with `cost heads`


[Document(page_content='Sensitivity to a one percentage point changes in the discount rate or the terminal growth rate and based on the results; management \nbelieves that no reasonably possible change in any of the above mentioned key assumptions would cause the carrying value to exceed the \nrecoverable amount.\n27 DUE TO FINANCIAL INSTITUTIONS\n2018\nAED‘0002017\nAED‘000\nCurrent accounts 878,693 1,538,954\nInvestment deposits     3,203,909     2,149,604\n4,082,602 3,688,558\nCurrent account – Central Bank of UAE          55,652                    -\n    4,138,254     3,688,558', metadata={'page': 105, 'source': '/home/fasih/ADIB/annual_report/ADIB Annual Report 2018.pdf'}), Document(page_content='been discounted at a rate of 4.5% (2019: 5.7%).\nSensitivity to a one percentage point changes in the discount rate or the terminal growth rate and based on the results; management \nbelieves

"Based on the available search results from the ADIB annual reports, the specific cost heads mentioned are as follows:\n\n1. Customer Financing Segment:\n   - Provision for impairment\n\n2. Investments Segment:\n   - Not mentioned in the available search results\n\n3. Retail Banking Segment:\n   - Not mentioned in the available search results\n\n4. Corporate Banking Segment:\n   - Not mentioned in the available search results\n\n5. Asset Management Segment:\n   - Not mentioned in the available search results\n\n6. Other Operating Segments:\n   - Property and equipment\n   - Depreciation\n\nPlease note that the search results may not cover all the cost heads mentioned in the ADIB annual reports. It's recommended to refer to the complete annual reports for a comprehensive understanding of the cost heads associated with each operating segment."

In [16]:
result = agent.run('what is the commission income in Abridged Income Statement under ADIB securities')
result



> Entering new AgentExecutor chain...

Invoking: `search_adib_annual_report` with `{'text': 'commission income', 'section': 'Abridged Income Statement', 'subsection': 'ADIB securities'}`




ToolException: Too many arguments to single-input tool search_adib_annual_report. Args: ['commission income', 'Abridged Income Statement', 'ADIB securities']

In [17]:
result = agent.run('Tell me about Burooj Properties LLC.')
result



> Entering new AgentExecutor chain...

Invoking: `search_adib_annual_report` with `{'query': 'Burooj Properties LLC'}`


[Document(page_content='Investment properties 22 - 625\nDevelopment properties 23 - 31,148\nOthers     15,753   100,758\n954,399 1,314,112\nThe above provision for impairment includes nil (2020: AED 31,773 thousand) pertaining to Burooj Properties LLC, a real estate subsidiary of \nthe Bank.', metadata={'page': 105, 'source': '/home/fasih/ADIB/annual_report/ADIB Annual Report 2021.pdf'}), Document(page_content='Burooj Properties LLC ** UAE 100 Real Estate Investments Not consolidated Fully consolidated\nMPM Properties LLC ** UAE 100 Real Estate Services Not consolidated Fully consolidated\nKawader Services LLC ** UAE 100 Manpower Supply Not consolidated Fully consolidated\nADIB (UK) Limited United Kingdom 100 Other Services Fully consolidated Fully consolidated\nADIB Holdings (Jersey) Ltd* (under liquidation)British Channel \nIslands  - Special Purpose Vehicle Full

"Burooj Properties LLC is a real estate subsidiary of Abu Dhabi Islamic Bank (ADIB). It is mentioned in the ADIB annual reports. Here is some information about Burooj Properties LLC:\n\n1. Provision for Impairment: The annual report states that the provision for impairment includes nil (2020: AED 31,773 thousand) pertaining to Burooj Properties LLC.\n\n2. Real Estate Investments: Burooj Properties LLC is involved in real estate investments. The report mentions investment properties and development properties associated with Burooj Properties LLC.\n\n3. Historical Activities: In the 2018 annual report, it is mentioned that due to the conditions in the real estate market in the UAE, ADIB deemed it prudent to increase impairments relating to Burooj's historical activities by a further AED 61.8 million in 2018.\n\nPlease note that the information provided is based on the available annual reports and may not represent the complete scope of Burooj Properties LLC's activities. For more detail

In [21]:
result = agent.run('what were the investments in 2023 in csv ')
result



> Entering new AgentExecutor chain...

Invoking: `search_adib_annual_report` with `{'text': 'investments'}`


[Document(page_content='properties. Investment properties are measured at cost less any accumulated depreciation and any accumulated impairment losses. \nDepreciation is charged on a straight-line basis over the assets’ estimated useful lives. The useful life of buildings is estimated to be 25 - 40 \nyears.\nInvestment properties are derecognized when either they have been disposed of or when the investment properties are permanently \nwithdrawn from use and no future economic benefits are expected from their disposal. Any gains or losses on the retirement or disposal of an \ninvestment property are recognized in the consolidated income statement.', metadata={'page': 99, 'source': '/home/fasih/ADIB/annual_report/ADIB Annual Report 2021.pdf'}), Document(page_content='The Group has entered into commercial property lease arrangements on its investment property portfolio. The Gro

[Document(page_content='properties. Investment properties are measured at cost less any accumulated depreciation and any accumulated impairment losses. \nDepreciation is charged on a straight-line basis over the assets’ estimated useful lives. The useful life of buildings is estimated to be 25 - 40 \nyears.\nInvestment properties are derecognized when either they have been disposed of or when the investment properties are permanently \nwithdrawn from use and no future economic benefits are expected from their disposal. Any gains or losses on the retirement or disposal of an \ninvestment property are recognized in the consolidated income statement.', metadata={'page': 99, 'source': '/home/fasih/ADIB/annual_report/ADIB Annual Report 2021.pdf'}), Document(page_content='The Group has entered into commercial property lease arrangements on its investment property portfolio. The Group has determined, based \non an evaluation of the terms and conditions of the arrangements, that it retains all

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4130 tokens (3991 in the messages, 139 in the functions). Please reduce the length of the messages or functions.